In [6]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 76.5 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Import libraries
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

import numpy as np
import rasterio
from rasterio.transform import from_origin

pd.set_option('display.max_columns', None)

In [4]:
# set data dir
DATA_DIR = Path('/content/drive/Shareddrives/ZINDI Data Science/1. Competitions/Amini/Challenge - Multiclass/FRONTEND')
os.listdir(DATA_DIR)

['Train.csv',
 'Test.csv',
 'SampleSubmission.csv',
 'multiclass_benchmark.csv',
 'StarterNotebook_Multiclass.ipynb']

In [5]:
# Load files
train = pd.read_csv(DATA_DIR / 'Train.csv')
test = pd.read_csv(DATA_DIR / 'Test.csv')
sample_submission = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')

# Preview train file
train.head()

,ID,time,Green,Blue,RED,NIR,SWIR1,SWIR2,Red_Edge,Aerosols,Red_Edge_2,Red_Edge_3,Red_Edge_4,Water_vapor,Cirrus,NDVI,NDMI,NDWI,CI,Target
0,1D_0000,2021-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1D_0000,2021-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1D_0000,2021-01-14,0.0845,0.1030,0.0583,0.1977,0.1152,0.0464,0.0742,0.1326,0.1682,0.2243,0.2500,0.0347,0.0007,0.544531,0.263663,-0.401134,0.120000,0
3,1D_0000,2021-01-19,0.1095,0.1357,0.0858,0.1596,0.0782,0.0330,0.0873,0.1620,0.1396,0.1736,0.1820,0.0260,0.0011,0.300733,0.342304,-0.186176,0.008666,0
4,1D_0000,2021-01-24,0.1128,0.1300,0.0919,0.2141,0.1240,0.0608,0.1035,0.1546,0.1845,0.2302,0.2606,0.0703,0.0041,0.399346,0.266489,-0.309881,0.059365,0


View on row of the data as an RGB image, what happens if you add more layers to this image? Does this give you an idea of how tabular data can be turned into image data and vice versa?

In [ ]:
# Preview test file
test.head()

,ID,time,Green,Blue,RED,NIR,SWIR1,SWIR2,Red_Edge,Aerosols,Red_Edge_2,Red_Edge_3,Red_Edge_4,Water_vapor,Cirrus,NDVI,NDMI,NDWI,CI
0,1D_0005,2021-01-04,0.1199,0.1391,0.0978,0.2329,0.1782,0.0989,0.1146,0.2127,0.2074,0.2675,0.3081,0.0503,0.0011,0.408527,0.133058,-0.320295,0.079096
1,1D_0005,2021-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1D_0005,2021-01-14,0.0818,0.1010,0.0563,0.1848,0.1139,0.0459,0.0756,0.1324,0.1687,0.2246,0.2513,0.0372,0.0007,0.532974,0.237362,-0.386347,0.146323
3,1D_0005,2021-01-19,0.1270,0.1581,0.1031,0.1899,0.1186,0.0583,0.1124,0.1763,0.1639,0.1987,0.2156,0.0298,0.0012,0.296246,0.231118,-0.198485,0.043155
4,1D_0005,2021-01-24,0.1122,0.1311,0.0916,0.2125,0.1259,0.0608,0.1060,0.1553,0.1840,0.2307,0.2611,0.0709,0.0046,0.397567,0.255910,-0.308901,0.072874


In [ ]:
# Preview sample submission
sample_submission.head()

,ID,Target
0,1D_0005,0
1,1D_000A,0
2,1D_000D,0
3,1D_000E,0
4,1D_0015,0


In [ ]:
# Check shapes of files
train.shape, test.shape, sample_submission.shape

((11544707, 20), (4946879, 19), (16960, 2))

In [ ]:
# Check for duplicates and missing values - train
train.duplicated().any(), train.isnull().sum().any() # Handle missing values by filling them with meadian or any other techniques

(False, True)

In [ ]:
# Check for duplicates and missing values - test
test.duplicated().any(), test.isnull().sum().any() # Handle missing values by filling them with meadian or any other techniques

(False, True)

In [ ]:
# Check unique ids per dataset
train.ID.nunique(), test.ID.nunique()

(39571, 16960)

In [ ]:
# A single ID represents a unique pixel over different time periods
# create a targets_df

train_targets_df = train.groupby('ID')['Target'].first().reset_index()
train_targets_df.head()

,ID,Target
0,1D_0000,0
1,1D_0001,0
2,1D_0002,0
3,1D_0003,0
4,1D_0004,0


In [ ]:
# Check data distribution
train_targets_df.Target.value_counts(normalize = True)

,proportion
Target,
0,0.433398
1,0.335119
2,0.231483


In [ ]:
train.head()

,ID,time,Green,Blue,RED,NIR,SWIR1,SWIR2,Red_Edge,Aerosols,Red_Edge_2,Red_Edge_3,Red_Edge_4,Water_vapor,Cirrus,NDVI,NDMI,NDWI,CI,Target
0,1D_0000,2021-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1D_0000,2021-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1D_0000,2021-01-14,0.0845,0.1030,0.0583,0.1977,0.1152,0.0464,0.0742,0.1326,0.1682,0.2243,0.2500,0.0347,0.0007,0.544531,0.263663,-0.401134,0.120000,0
3,1D_0000,2021-01-19,0.1095,0.1357,0.0858,0.1596,0.0782,0.0330,0.0873,0.1620,0.1396,0.1736,0.1820,0.0260,0.0011,0.300733,0.342304,-0.186176,0.008666,0
4,1D_0000,2021-01-24,0.1128,0.1300,0.0919,0.2141,0.1240,0.0608,0.1035,0.1546,0.1845,0.2302,0.2606,0.0703,0.0041,0.399346,0.266489,-0.309881,0.059365,0


In [ ]:
# Aggrgate pixels over the different time periods to get a wholesome view
# Get columns to aggregate
agg_cols = train.columns.tolist()[2:-1]
agg_cols

['Green',
 'Blue',
 'RED',
 'NIR',
 'SWIR1',
 'SWIR2',
 'Red_Edge',
 'Aerosols',
 'Red_Edge_2',
 'Red_Edge_3',
 'Red_Edge_4',
 'Water_vapor',
 'Cirrus',
 'NDVI',
 'NDMI',
 'NDWI',
 'CI']

What are these bands? Are there spefic bands used choose over another?

## Feature Engineering

What features or aggregations would you create?

Consider the timeseries component of the data

## Modeling

What is the best model or models you can you use for this type of problem?

Is there only one model? Would an ensemble of models add something?

What happens if you take into consideration run size and model implementation, what would be the lightest, fastest and most accurate model?

## Making your submission

Here is your code to help you prepare the data for a submission to Zindi. Note you will need to edit this code to the dataframe names you used in your modeling building.

In [ ]:
# Prepare test data
X_test = test.drop(['ID'], axis = 1)

# Make predictions on test
test_preds = model.predict(X_test)

# Prepares sub file
sub = pd.DataFrame()
sub['ID'] = test.ID
sub['Target'] = test_preds
sub.head()

,ID,Target
0,1D_0005,0
1,1D_000A,0
2,1D_000D,0
3,1D_000E,0
4,1D_0015,0


In [ ]:
# Create submission to be uploaded to Zindi for scoring
sub.to_csv('Submission.csv', index = False)

## Best of luck!

Remember their is only learning on Zindi!